# <center> Parse J1939 Multi Frame Messages
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" />

## Prerequisites
Successfully run [01_TestCAN_RX.ipynb](01_TestCAN_RX.ipynb).

In this exercise, we will look for VIN and Component ID, which are long ascii strings.

A good file for this exercise is one from a diagnostics session.

https://www.engr.colostate.edu/~jdaily/J1939/files/candump_dgdiagnostics_paccar_mx.txt

In Linux we can download this file from the command prompt:
```
wget https://www.engr.colostate.edu/~jdaily/J1939/files/candump_dgdiagnostics_paccar_mx.txt
```

To set replay up for this file on a virtual can channel with Linux Socket CAN:
```
sudo ip link add dev vcan3 type vcan
sudo ip link set vcan3 up
canplayer -l i -I candump_dgdiagnostics_paccar_mx.txt vcan3=can1 &
```

In [1]:
#!/usr/bin/python3
import socket
import struct

# Open a socket and bind to it from SocketCAN
sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
interface = "vcan3"

# Bind to the interface
sock.bind((interface,))

# To match this data structure, the following struct format can be used:
can_frame_format = "<lB3x8s"

In [2]:
#Make a CAN reading function
def unpack_CAN(can_packet,display=False):
    can_id, can_dlc, can_data = struct.unpack(can_frame_format, can_packet)
    extended_frame = bool(can_id & socket.CAN_EFF_FLAG)
    if extended_frame:
        can_id &= socket.CAN_EFF_MASK
        can_id_string = "{:08X}".format(can_id)
    else: #Standard Frame
        can_id &= socket.CAN_SFF_MASK
        can_id_string = "{:03X}".format(can_id)
    if display:
        hex_data_string = ' '.join(["{:02X}".format(b) for b in can_data[:can_dlc]])
        print("{} {} [{}] {}".format(interface, can_id_string, can_dlc, hex_data_string))
    return can_id, can_dlc, can_data[:can_dlc]

In [3]:
#parse J1939 protocol data unit information from the ID using bit masks and shifts
PRIORITY_MASK = 0x1C000000
EDP_MASK      = 0x02000000
DP_MASK       = 0x01000000
PF_MASK       = 0x00FF0000
PS_MASK       = 0x0000FF00
SA_MASK       = 0x000000FF
PDU1_PGN_MASK = 0x03FF0000
PDU2_PGN_MASK = 0x03FFFF00

def get_j1939_from_id(can_id):
    #priority
    priority = (PRIORITY_MASK & can_id) >> 26

    #Extended Data Page
    edp = (EDP_MASK & can_id) >> 25
    
    # Data Page
    dp = (DP_MASK & can_id) >> 24
    
    # Protocol Data Unit (PDU) Format
    PF = (can_id & PF_MASK) >> 16
    
    # Protocol Data Unit (PDU) Specific
    PS = (can_id & PS_MASK) >> 8
    
    # Determine the Parameter Group Number and Destination Address
    if PF >= 0xF0: #240

        # PDU 2 format, include the PS as a group extension
        DA = 255
        PGN = (can_id & PDU2_PGN_MASK) >> 8
    else:
        PGN = (can_id & PDU1_PGN_MASK) >> 8
        DA = PS
    # Source address
    SA = (can_id & SA_MASK)
    
    return priority,PGN,DA,SA

In [4]:
# Read and process some used for transport layer messages.
samples = 150
i=0
while i < samples:
    # Read the message from the newtork
    can_packet = sock.recv(16)
    #Parse the bytes into a CAN message
    can_id, can_dlc, can_data = unpack_CAN(can_packet)
    #Parse the CAN ID into J1939
    priority,pgn,da,sa = get_j1939_from_id(can_id)
    
    if pgn == 0xEA00 or pgn == 0xEB00 or pgn == 0xEC00:
        i+=1
        print(priority,pgn,da,sa,can_data) 
    

7 60160 255 0 b'\x19\t\x01\x84\x06\t\x01\x84'
7 60160 255 0 b'\x1a\x06\t\x01u\x02\x05\x01'
7 60160 255 0 b'\x1b\xf9\x13\t\x01\xfa\x13\t'
7 60160 255 0 b'\x1c\x01\xff\xff\xff\xff\xff\xff'
7 60416 255 0 b' \xbe\x00\x1c\xff\xca\xfe\x00'
7 60160 255 0 b'\x01W\xff\x9d\x00\x03\x01\xfb'
7 60160 255 0 b'\x02\x06\x0b~J\x00\x0e~'
7 60160 255 0 b'\x03\xe1\x06\x0e~f\x00\x04'
7 60160 255 0 b'\x04\x01Q\x00\x04\x01\xb9\x04'
7 60160 255 0 b'\x05\x04\x01i\x00\x03\x01F'
7 60160 255 0 b'\x06\n\x03\x01\xab\x00\x03\x01'
7 60160 255 0 b'\x07\xcf\x04\x02\x01\xd3\x04\x02'
7 60160 255 0 b'\x08\x01\x84\x06\x02\x01n\x00'
7 60160 255 0 b'\t\x03\x01^\x00\x04\x01\xae'
7 60160 255 0 b'\n\x00\x03\x01\xf5\x13\x13\x01'
7 60160 255 0 b'\x0b\x91\x0c\x13\x01a\x02\t'
7 60160 255 0 b'\x0c\x01\x7f\x02\t\x01\x84\x06'
7 60160 255 0 b'\r\t\x01\x84\x06\t\x01\xf8'
7 60160 255 0 b'\x0e\x13\t\x01\x84\x06\t\x01'
7 60160 255 0 b'\x0f\x84\x06\t\x01\x84\x06\t'
7 60160 255 0 b'\x10\x01\x84\x06\t\x01\xfd\x13'
7 60160 255 0 b'\x11\t\x01\x

Let's look for Component ID: SPN 237, PGN 65259. 

First, we need to understand how these messages are being broadcast. Often these messages are broadcast based on a request. 

### J1939 Request
The request message is one of the few J1939 messages that are less than 8 bytes. 
It is specified as follows:

Rate: 2 or 3 times per second.

PGN: 59904 (0xEA00)


Priority: 6

PS: Destination address, which is either global (0xFF) or specific

The first three bytes is the PGN being requeste in reverse byte order (Big Endian)

Example:

vcan3 18EA0FF9#EBFE00


In [5]:
struct.unpack("<H",b'\xEB\xFE')[0]

65259

This is a request for Component ID, which is an ASCII string with the following format:
```
MAKE*MODEL*SERIALNUMBER*UNITNUMBER
```
We need to get multiple messages to put all this together.

## J1939 Transport Protocol (TP)
There are 2 messages that make up the commonly used J1939 Transport Protocol. These are PDU1 messages, which means they are logically point-to-point (not physically, since all nodes see all messages). These messages are as follows:

### PGN 60416 (0xEC00) -  Connection Management
Transport Protocol-Connection Management (TP.CM)

Definition: Used for the transfer of Parameter Groups that have 9 bytes or more of data.

Default priority: 7

Data ranges for parameters used by this Group Function:

Control byte: One of the following 16,17,19,32,255.

Total Message Size, number of bytes: 9 to 1785 (2 bytes)

Total Number of Packets: 2 to 255 (1 byte)

Maximum Number of Packets: 2 to 255 (1byte)

Number of Packets that can be sent: 0 to 255 (1 byte)

Next Packet Number to be Sent: 1 to 255 (1 byte)

Sequence Number: 1 to 255 (1 byte)

#### Connection Mode: BAM
Byte 0: Control byte = 32 (0x20), Broadcast Announce Message (BAM)

Bytes 1,2: Total message size, number of bytes (Big Endian)

Byte 3: Total number of packets

Byte 4: Reserved (0xFF)

Bytes 5,6,7: Parameter Group Number of the packeted message (Big Endian)

Note: The destination on a BAM is often 255 for all the nodes.

#### Connection Mode: RTS
Byte 0: Control byte = 16 (0x10), Destination Specific Request To Send (RTS)

Bytes 1,2: Total message size, number of bytes (Big Endian)

Byte 3: Total number of packets

Byte 4: Maximum number of packets that can be sent in response to one CTS. 0xFF indicates that
no limit exists for the originator.

Bytes 5,6,7: Parameter Group Number of the packeted message (Big Endian)

Note: Only to be transmitted by the originator.

#### Connection Mode: CTS
Byte 0: Control byte = 17 (0x11), Destination Specific Clear To Send (CTS)

Bytes 1: Number of packets that can be sent. This value shall be no larger than the smaller of the
two values in byte 3 and byte 4 of the RTS message.

Byte 2: The next packet number to be sent

Byte 3-4: 0xFF (not used)

Bytes 5,6,7: Parameter Group Number of the packeted message (Big Endian)

Note: Only to be transmitted by the responder.

#### Connection Mode: ACK
Byte 0: Control byte = 19 (0x13), Destination Specific End of Message Acknowledgement (ACK)

Bytes 1,2: Total message size, number of bytes (Big Endian)

Byte 3: Total number of packets

Byte 4: Reserved (0xFF)

Bytes 5,6,7: Parameter Group Number of the packeted message (Big Endian)

Note: Only to be transmitted by the responder.

#### Connection Mode: Abort
Byte 0: Control byte = 255 (0xFF), Connection Abort

Bytes 1: Connection Abort reason

Byte 2-4: Reserved (0xFF)

Bytes 5,6,7: Parameter Group Number of the packeted message (Big Endian)

Abort Reasons:
```
1 - Already in one or more connection managed sessions and cannot support another.
2 - System resources were needed for another task.
3 - A timeout occurred and this is the connection abort to close the session.
4 - CTS messages received when data transfer is in progress.
5 - Maximum retransmit request limit reached
```
### PGN 60160 (0xEB00) - Data Transfer
Parameter Group Name: Transport Protocol-Data Transfer (TP.DT)

Definition: Used for the transfer of data associated with Parameter Groups that have more than 8 bytes
of data.

Default priority: 7

Data ranges for parameters used by this Group Function:

Byte 0: Sequence Number (1-255)

Bytes 1-7: packetized data. Fill with 0xFF at end to make all 8 bytes. 

In [6]:
total_possible_bytes = 7*255
print("J1939 can transmit up to {} bytes using the transport protocol.".format(total_possible_bytes))

J1939 can transmit up to 1785 bytes using the transport protocol.


Since simultaneous transport protocol sessions can take place at once, we can use a dictionary to  handle simultaneous sessions.

In [6]:
# J1939 transport class
transport_messages = {}
def read_message(can_packet):
    #Parse the bytes into a CAN message
    can_id, can_dlc, can_data = unpack_CAN(can_packet,display=False)
    #Parse the CAN ID into J1939
    priority,pgn,da,sa = get_j1939_from_id(can_id)

    if pgn == 60416: #(0xEC00) - Connection Management is_first_frame(message_data):
        #Check the control byte:
        transport_pgn = (can_data[7] << 16) + (can_data[6] << 8) + can_data[5]
        if can_data[0] == 32: #BAM
            #setup a dictionary with the source and destination pair
            #print("Found transport message for BAM of PGN {} from Source {}".format(transport_pgn,sa))
            total_bytes = struct.unpack("<H",can_data[1:3])[0]
            total_messages = can_data[3]
            transport_messages[(sa,da)] = {'pgn':transport_pgn,
                                           'total_messages':total_messages,
                                           'total_bytes':total_bytes,
                                           'received_messages': {}}
        elif can_data[0] == 255: #abort
            transport_messages[(sa,da)] = {}
        else:
            pass
    elif pgn == 60160: #(0xEB00) - Data Transfer
        i = can_data[0]
        packetized_data=can_data[1:8]
        try:
            transport_messages[(sa,da)]['received_messages'][i] = packetized_data          
        except KeyError:
            return (None, None, None, None, None)
        if len(transport_messages[(sa,da)]['received_messages']) == transport_messages[(sa,da)]['total_messages']:
            return_bytes=b''
            #print(transport_messages[sa]['received_messages'])
            for k,v in sorted(transport_messages[(sa,da)]['received_messages'].items()):
                return_bytes += v
            return (priority,
                    transport_messages[(sa,da)]['pgn'],
                    da,
                    sa,
                    return_bytes[:transport_messages[(sa,da)]['total_bytes']])
        else:
            pass
        
    else:
        return priority,pgn,da,sa,can_data[:can_dlc]
    return (None, None, None, None, None)


In [8]:
# Read and process some used for transport layer messages.
samples = 2
i=0
while i < samples:
    # Read the message from the newtork
    can_packet = sock.recv(16)
    #Parse the bytes into a J1939 message
    priority,pgn,da,sa,can_data = read_message(can_packet)
    if can_data is None:
        continue
    if len(can_data) > 8 and pgn == 0xffff:
        i+=1
        
        print("PGN {} from SA {}:\n{}\n".format(pgn,sa,can_data) )
        print(can_data.decode('utf-8'))

PGN 65535 from SA 249:
b'Vehicle Communication is Awesome'

Vehicle Communication is Awesome
PGN 65535 from SA 249:
b'Vehicle Communication is still Awesome'

Vehicle Communication is still Awesome


## Assignment:
Write a parser to handle Request to Send and Clear to Send Messages.